In [ ]:
import adi
import matplotlib.pyplot as plt
import numpy as np
import scipy
from IPython import display

from ofdm_transceiver import ** 

In [ ]:
sample_rate = 5e6  
center_freq = 1.5e9

delta_t = 1 / sample_rate
delta_f = 30e3 
frac_guard = 0.5

N_sc_av = int(sample_rate / delta_f)
N_fft =  2**(int(np.log2(N_sc_av))) 
N_sc_use = int( N_fft * frac_guard)
guard_length = int( 0.5 * N_fft )
CP_len = int(N_fft * 0.2 ) 
num_bits_sym = 1

BlockSize = N_sc_use * num_bits_sym


In [ ]:
do_cfo_corr = 1
do_ce = 1
dc_offset = 0


np.random.seed(123)
preamble = create_preamble(N_fft, CP_len, 2)

In [ ]:
pilot, pilot_freq_ = create_data(N_sc_use, N_fft, CP_len, aditional_return = 'mod_sym_pilot')
data, data_stream = create_data(N_sc_use, N_fft, CP_len, aditional_return = 'data_stream')
guard = np.zeros((guard_length, 1), dtype=np.complex64)
frame = np.concatenate((preamble, pilot, data, guard))

repeated_frame = np.tile(frame, reps = (3,1))

frame_len = len(frame)
preamble_len = N_fft//2 



sdr = adi.ad9361(uri='ip:192.168.1.1')
samp_rate = sample_rate    # must be <=30.72 MHz if both channels are enabled
num_samps = len(repeated_frame)      # number of samples per buffer.  Can be different for Rx and Tx
rx_lo = int(center_freq)
rx_mode = "slow_attack"  # can be "manual" or "slow_attack"
rx_gain0 = 70
rx_gain1 = 70
tx_lo = rx_lo
tx_gain0 = -10
tx_gain1 = -10

sdr.rx_enabled_channels = [0, 1]
sdr.sample_rate = int(samp_rate)
sdr.rx_lo = int(rx_lo)
sdr.gain_control_mode = rx_mode
sdr.rx_hardwaregain_chan0 = int(rx_gain0)
#sdr.rx_hardwaregain_chan1 = int(rx_gain1)
sdr.rx_buffer_size = int(num_samps)

while(True):
    data = sdr.rx()
    print(data[0].shape)
    rx_sig = data[0] + data[1]
    rx_len = len(rx_sig) 

    idx_max, corr_value = find_edges(rx_sig, frame_len, preamble_len, start_idx = 0)
    frame_receive = rx_sig[idx_max : idx_max + frame_len]

    frame_receive = cfo(frame_receive, corr_value, preamble_len) if do_cfo_corr else frame_receive
    pilot_receive = frame_receive[N_fft + CP_len : 2*N_fft + CP_len]

    pilot_freq = np.fft.fft(pilot_receive, N_fft, 0, norm="ortho")
    rec_sym_pilot = baseband_freq_domian(pilot_freq, N_sc_use)

    # apply CE
    h_ls = rec_sym_pilot[:, 0] / pilot_freq_[:,0]
    h_ls = channel_estimation(h_ls, CP_len, N_fft) if do_ce else h_ls

    rec_data_sym = frame_receive[2 * (N_fft + CP_len) : 2 * (N_fft + CP_len) + N_fft]
    rec_data_sym_freq = np.fft.fft(rec_data_sym, N_fft, 0, norm="ortho")
    rec_sym_data = baseband_freq_domian(rec_data_sym_freq, BlockSize)
    eq_data = rec_sym_data[:, 0] / h_ls


    bit_arr = demodulate(eq_data, N_sc_use)
    ber = get_ber(data_stream, bit_arr, N_sc_use)
    SNR_est = estimate_SNR(pilot_freq, rec_sym_pilot, N_sc_use)

    print(data_stream[:10,0])
    print(bit_arr[:10])
    print(f'SNRest={SNR_est}')
    print(f'BER={ber}')
